<a href="https://colab.research.google.com/github/jinholeetopology/ml/blob/master/[%EC%8B%A4%EC%8A%B5]_Logistic_Regression%EC%9C%BC%EB%A1%9C_%EC%88%AB%EC%9E%90_%EB%B6%84%EB%A5%98%EA%B8%B0%EB%A5%BC_%EA%B5%AC%ED%98%84%ED%95%98%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Logistic regression with MNIST

In [2]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data
print("packages loaded")

packages loaded


In [3]:
sess = tf.Session()
print('OPEN SESSION')

OPEN SESSION


## Download and extract MNIST dataset

In [21]:
mnist      = input_data.read_data_sets('data/',one_hot = True)
training   = mnist.train.images
trainlabel = mnist.train.labels
testing    = mnist.test.images
testlabel  = mnist.test.labels
print("MNIST loaded")

Extracting data/train-images-idx3-ubyte.gz
Extracting data/train-labels-idx1-ubyte.gz
Extracting data/t10k-images-idx3-ubyte.gz
Extracting data/t10k-labels-idx1-ubyte.gz
MNIST loaded


## Create tensor graph for Logistic regression

In [0]:
x = tf.placeholder("float", [None, 784])
y = tf.placeholder("float", [None, 10])
W = tf.Variable(tf.zeros([784,10]))
b = tf.Variable(tf.zeros([10]))

# Logistic regression model
actv = tf.nn.softmax(tf.matmul(x,W) + b) # 행렬 x와 W 를 곱해서, b 를 더한 결과는 [1,10] 행렬이 되는데 이것을 softmax 함수로 보낸 결과

# Cost function
cost = tf.reduce_mean(-tf.reduce_sum(y*tf.log(actv), reduction_indices = 1)) # axis 의 옛날 이름

# Optimizer
learning_rate = 0.01
optm = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost) # gradient descent 를 이용하여 cost 를 minimize 하는 방향으로 학습

## tf.reduce_mean 정리

In [9]:
# tf.reduce_mean 에서 reduction_indices = 1 의미
 # 1. reduction_indices 는 axis 의 옛날 이름이다(tf 공식 홈페이지 내용 : reduction_indices: The old (deprecated) name for axis)


x1 = tf.constant([[ 1., 2.], [3., 4.]])
print(sess.run(tf.reduce_mean(x1)))
print(sess.run(tf.reduce_mean(x1,0))) # 0 은 열마다 평균을 의미. 즉, 1 과 3 의 평균이므로 2, 2 와 4의 평균 3 이 나옴
print(sess.run(tf.reduce_mean(x1,1))) # 1 은 열마다 평균을 의미. 즉, 1 과 2 의 평균이므로 1.5, 3 와 4의 평균 3.5 가 나옴

2.5
[2. 3.]
[1.5 3.5]


## tf.reduce_sum 정리

In [10]:
# tf.reduce_sum 예제

x = tf.constant([[1., 2.], [3., 8.]])
a = tf.reduce_sum(x)    # 모든 element 를 다 더하기
b = tf.reduce_sum(x, 0) # x 에서 0 번째 element 인 1 과 3 을 더하기
c = tf.reduce_sum(x, 1) # x 에서 1 번째 element 인 1 과 3 을 더하기

sess = tf.Session()
sess.run(tf.global_variables_initializer())

print( "a=", sess.run(a) )
print( "b=", sess.run(b) )
print( "c=", sess.run(c) )

a= 14.0
b= [ 4. 10.]
c= [ 3. 11.]


In [11]:
# tf.reduce_mean 예제

x = tf.constant([[1., 2.], [3., 8.]])
a = tf.reduce_mean(x)    # 모든 element 의 평균((1+2+3+8)/4 = 3.5
b = tf.reduce_mean(x, 0)
c = tf.reduce_mean(x, 1)

sess = tf.Session()
sess.run(tf.global_variables_initializer())

print( "a=", sess.run(a) )
print( "b=", sess.run(b) )
print( "c=", sess.run(c) )

a= 3.5
b= [2. 5.]
c= [1.5 5.5]


## Preductui and Accuracy

In [0]:
# Prediction
pred = tf.equal(tf.argmax(actv,1), tf.argmax(y,1))

# Accuracy
accr = tf.reduce_mean(tf.cast(pred,"float"))

# Initialized
init = tf.global_variables_initializer()

## tf.equal 정리

In [13]:
# tf.equal(x, y) : x, y를 비교하여 boolean 값을 반환

x1 = tf.constant(1)
y1 = tf.constant(1)

prediction = tf.equal(x1,y1)

print(sess.run(prediction))

True


## tf.case : 원하는 data type 으로 변환

In [14]:
# 위에서 prediction 은 True 라는 boolean 타입의 data 이다. 이것을 float 함수를 이용하여 숫자로 바꿀 수 있다.

p = sess.run(tf.cast(prediction,"float"))
print(p)

1.0


## argmax 정리

In [15]:
# tf.argmax(X, a) : X 배열에서 가장 큰 값을 ‘열(1)’ 기준 몇번째인지 index를 반환

x1 = [0,0,1,0]
tf.argmax(x1).eval(session=sess)

2

In [16]:
x1 = [[1,2],[3,4]]
tf.argmax(x1,1).eval(session=sess)

array([1, 1])

## Train model

In [24]:
training_epochs = 50    # 총 50번 학습하자
batch_size      = 100   # batch size 는 100 이므로 하자
display_step    = 5

# Session
sess = tf.Session()
sess.run(init)

# mini-batch learning

for epoch in range(training_epochs):
  avg_cost = 0.
  num_batch = int(mnist.train.num_examples/batch_size)
  for i in range(num_batch):
    batch_xs, batch_ys = mnist.train.next_batch(batch_size)
    sess.run(optm, feed_dict={x:batch_xs, y:batch_ys})
    feeds = {x:batch_xs, y:batch_ys}
    avg_cost += sess.run(cost, feed_dict=feeds)/num_batch
    
  # display
    
  if epoch % display_step == 0:
    feeds_train = {x:batch_xs, y:batch_ys}
    feeds_test = {x:mnist.test.images, y:mnist.test.labels}
    train_acc = sess.run(accr, feed_dict=feeds_train)
    test_acc  = sess.run(accr, feed_dict=feeds_test)
    print("Epoch: [%d/%d] cost: [%f] train_acc: [%f] test_acc: [%f]" % (epoch, training_epochs, avg_cost, train_acc, test_acc))

print("DONE")

Epoch: [0/50] cost: [1.176905] train_acc: [0.890000] test_acc: [0.856100]
Epoch: [5/50] cost: [0.440961] train_acc: [0.860000] test_acc: [0.894800]
Epoch: [10/50] cost: [0.383336] train_acc: [0.880000] test_acc: [0.904900]
Epoch: [15/50] cost: [0.357327] train_acc: [0.910000] test_acc: [0.909000]
Epoch: [20/50] cost: [0.341431] train_acc: [0.930000] test_acc: [0.912400]
Epoch: [25/50] cost: [0.330525] train_acc: [0.920000] test_acc: [0.915000]
Epoch: [30/50] cost: [0.322370] train_acc: [0.970000] test_acc: [0.915800]
Epoch: [35/50] cost: [0.315940] train_acc: [0.930000] test_acc: [0.917300]
Epoch: [40/50] cost: [0.310747] train_acc: [0.910000] test_acc: [0.917800]
Epoch: [45/50] cost: [0.306377] train_acc: [0.910000] test_acc: [0.918100]
DONE


## L^2 regulization 과 optimization = Adam 을 사용해 보기

In [0]:
x = tf.placeholder("float", [None, 784])
y = tf.placeholder("float", [None, 10])
W = tf.Variable(tf.zeros([784,10]))
b = tf.Variable(tf.zeros([10]))

# Logistic regression model
actv = tf.nn.softmax(tf.matmul(x,W) + b) # 행렬 x와 W 를 곱해서, b 를 더한 결과는 [1,10] 행렬이 되는데 이것을 softmax 함수로 보낸 결과

# Cost function
cost = tf.reduce_mean(-tf.reduce_sum(y*tf.log(actv), reduction_indices = 1)) # axis 의 옛날 이름

# Optimizer
learning_rate = 0.01
optm = tf.train.AdamOptimizer(learning_rate).minimize(cost) # gradient descent 를 이용하여 cost 를 minimize 하는 방향으로 학습

In [26]:
training_epochs = 50
batch_size      = 100
display_step    = 5

# Session
sess = tf.Session()
sess.run(init)

# mini-batch learning

for epoch in range(training_epochs):
  avg_cost = 0.
  num_batch = int(mnist.train.num_examples/batch_size)
  for i in range(num_batch):
    batch_xs, batch_ys = mnist.train.next_batch(batch_size)
    sess.run(optm, feed_dict={x:batch_xs, y:batch_ys})
    feeds = {x:batch_xs, y:batch_ys}
    avg_cost += sess.run(cost, feed_dict=feeds)/num_batch
    
  # display
    
  if epoch % display_step == 0:
    feeds_train = {x:batch_xs, y:batch_ys}
    feeds_test = {x:mnist.test.images, y:mnist.test.labels}
    train_acc = sess.run(accr, feed_dict=feeds_train)
    test_acc  = sess.run(accr, feed_dict=feeds_test)
    print("Epoch: [%d/%d] cost: [%f] train_acc: [%f] test_acc: [%f]" % (epoch, training_epochs, avg_cost, train_acc, test_acc))
    print("DONE")

FailedPreconditionError: ignored

In [0]:
class AdamOptimizer:
    def __init__(self, weights, alpha=0.001, beta1=0.9, beta2=0.999, epsilon=1e-8):
        self.alpha = alpha
        self.beta1 = beta1
        self.beta2 = beta2
        self.epsilon = epsilon
        self.m = 0
        self.v = 0
        self.t = 0
        self.theta = weights

    def backward_pass(self, gradient):
        self.t = self.t + 1
        self.m = self.beta1*self.m + (1 - self.beta1)*gradient
        self.v = self.beta2*self.v + (1 - self.beta2)*(gradient**2)
        m_hat = self.m/(1 - self.beta1**self.t)
        v_hat = self.v/(1 - self.beta2**self.t)
        self.theta = self.theta - self.alpha*(m_hat/(np.sqrt(v_hat) - self.epsilon))
        return self.theta